In [75]:
import numpy as np
import itertools
import re

In [76]:
registers = {}
memory = []
pointer = 0

with open('input.txt') as f:
    regex = 'Register ([ABC]): ([0-9]*)'
    for line in itertools.takewhile(lambda x: x != '\n', f):
        match = re.match(regex, line)
        registers[match.group(1)] = int(match.group(2))
    regex = 'Program: (.*)'
    for line in f:
        match = re.match(regex, line)
        for byte in match.group(1).split(','):
            memory.append(int(byte))

print(pointer)
print(memory)
print(registers)

0
[2, 4, 1, 3, 7, 5, 0, 3, 4, 1, 1, 5, 5, 5, 3, 0]
{'A': 45483412, 'B': 0, 'C': 0}


In [77]:
'''
    Combo operands 0 through 3 represent literal values 0 through 3.
    Combo operand 4 represents the value of register A.
    Combo operand 5 represents the value of register B.
    Combo operand 6 represents the value of register C.
    Combo operand 7 is reserved and will not appear in valid programs.
'''
def get_combo_operand(operand):
    if operand >= 0 and operand <= 3:
        return operand
    elif operand == 4:
        return registers['A']
    elif operand == 5:
        return registers['B']
    elif operand == 6:
        return registers['C']
    elif operand == 7:
        raise ValueError

In [78]:
def execute_instruction():
    global pointer
    opcode = memory[pointer]
    operand = memory[pointer+1]
    pointer += 2
    
    if opcode == 0 or opcode == 2 or opcode == 5 or opcode == 6 or opcode == 7:
        operand = get_combo_operand(operand)

    if opcode == 0:
        '''
        The adv instruction (opcode 0) performs division.
        The numerator is the value in the A register.
        The denominator is found by raising 2 to the power of the instruction's combo operand.
        (So, an operand of 2 would divide A by 4 (2^2); an operand of 5 would divide A by 2^B.)
        The result of the division operation is truncated to an integer and then written to the A register.
        '''
        registers['A'] = int(registers['A'] / (np.pow(2, operand)))
    elif opcode == 1:
        '''
        The bxl instruction (opcode 1) calculates the bitwise XOR of register B and the instruction's literal operand,
        then stores the result in register B.
        '''
        registers['B'] = registers['B'] ^ operand
    elif opcode == 2:
        '''
        The bst instruction (opcode 2) calculates the value of its combo operand modulo 8
        (thereby keeping only its lowest 3 bits), then writes that value to the B register.
        '''
        registers['B'] = operand % 8
    elif opcode == 3:
        '''
        The jnz instruction (opcode 3) does nothing if the A register is 0.
        However, if the A register is not zero, it jumps by setting the instruction pointer
        to the value of its literal operand; if this instruction jumps, the instruction pointer
        is not increased by 2 after this instruction.
        '''
        if registers['A'] != 0:
            pointer = operand
    elif opcode == 4:
        '''
        The bxc instruction (opcode 4) calculates the bitwise XOR of register B and register C,
        then stores the result in register B. (For legacy reasons, this instruction reads an operand
        but ignores it.)
        '''
        registers['B'] = registers['B'] ^ registers['C']
    elif opcode == 5:
        '''
        The out instruction (opcode 5) calculates the value of its combo operand modulo 8,
        then outputs that value. (If a program outputs multiple values, they are separated by commas.)
        '''
        outputs.append(operand % 8)
    elif opcode == 6:
        '''
        The bdv instruction (opcode 6) works exactly like the adv instruction except that the
        result is stored in the B register. (The numerator is still read from the A register.)
        '''
        registers['B'] = int(registers['A'] / (np.pow(2, operand)))
    elif opcode == 7:
        '''
        The cdv instruction (opcode 7) works exactly like the adv instruction except that the
        result is stored in the C register. (The numerator is still read from the A register.)
        '''
        registers['C'] = int(registers['A'] / (np.pow(2, operand)))

In [79]:
outputs = []
pointer = 0
while pointer < len(memory):
    execute_instruction()

print(",".join([str(output) for output in outputs]))

1,5,0,5,2,0,1,3,5
